In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
print(access_token)

APP_USR-6628000663056293-111307-0612588d50de258bbd5c60260f9c52b5-233632476


## Verificando dados da conta

In [3]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {
  'Authorization': f'Bearer {access_token}'
}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

# Encontre posição do início e fim do campo "id"
start_pos = response.find('"id"')  # Encontra onde começa "id"
end_pos = response.find(',', start_pos)  # Encontra onde termina "id"

# Extraia o valor de "id"
seller_id = response[start_pos:end_pos].split(":")[1]
seller_id = seller_id.strip().strip('"')

print(f'O valor de "seller_id" é: {seller_id}')

{"id":233632476,"nickname":"MUSICALCENTERPG","registration_date":"2016-11-05T17:35:04.000-04:00","first_name":"MUSICAL CENTER PONTA","last_name":"GROSSA EIRELI -","gender":"","country_id":"BR","email":"musicalc168@gmail.com","identification":{"number":"10885874000127","type":"CNPJ"},"address":{"address":"Estrada da Gruta 775","city":"Louveira","state":"BR-SP","zip_code":"13290431"},"phone":{"area_code":"19","extension":"","number":"38787702","verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"brand","tags":["brand","large_seller","eshop","mshops","messages_as_seller"],"logo":null,"points":4519,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/MUSICALCENTERPG","seller_experience":"ADVANCED","bill_data":{"accept_credit_note":"Y"},"seller_reputation":{"level_id":"5_green","power_seller_status":"platinum","transactions":{"canceled":1514,"completed":32748,"period":"historic","ratings":{"negative":0.05,"neutral":0.03,"positive":0.92},"t

## Carregando Inventory Ids de Produtos a serem pesquisados

Esses dados podem ser encontrados n coluna 'Código ML' da planilha 'Relação Full x Tiny' em 'Envios Full.xlsx'

In [4]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)

df_codes.head(1)


C:\Users\couti\AppData\Local\Temp\ipykernel_11900\1288080312.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
0,509517168,7899028808537,JFGN34621,MLB924922735,FULLP10NINJA10FTC,2023-11-10 16:32:30.451360,None


In [5]:
# Verifica se há duplicatas
duplicates = df_codes.duplicated()
n_dup = duplicates.sum()

print(f'Número de duplicatas: {n_dup}')

# Encontre as duplicatas no DataFrame
duplicates = df_codes[df_codes.duplicated(keep=False)]

# Exiba as próprias duplicatas
duplicates

Número de duplicatas: 0


,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at


In [6]:
# Remove as duplicatas e atualiza o DataFrame
df_codes = df_codes.drop_duplicates()

df_codes.shape

(878, 7)

In [7]:
df_codes.head(1)

,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
0,509517168,7899028808537,JFGN34621,MLB924922735,FULLP10NINJA10FTC,2023-11-10 16:32:30.451360,None


## Pegando Estoque de FulFillment

In [8]:
df_codes = df_codes.dropna(subset=['ml_inventory_id'])
df_codes['ml_inventory_id'].value_counts()

ml_inventory_id
NaN          118
BXUC56742      5
DMTU81740      5
KMCE82136      5
FSNB76403      5
            ... 
PNST71108      1
GLMK71067      1
VFWO70372      1
EDGX70669      1
UCQQ19395      1
Name: count, Length: 597, dtype: int64

In [9]:
codes = df_codes['ml_inventory_id'].unique()
print(len(codes))

597


In [10]:
df_codes['ml_inventory_id']

0      JFGN34621
1      ERIM51807
2      HVCS52763
3      ZQIV51979
4      RBUB53192
         ...    
873          NaN
874          NaN
875          NaN
876          NaN
877          NaN
Name: ml_inventory_id, Length: 878, dtype: object

In [11]:
len(codes)

597

In [12]:
valor_a_remover = 3
import numpy as np

codes = np.delete(codes, np.where(codes == 'NaN'))
codes

array(['JFGN34621', 'ERIM51807', 'HVCS52763', 'ZQIV51979', 'RBUB53192',
       'FXMN50419', 'TDFV51283', 'PKNN52962', 'MAEG54107', 'RYWV54124',
       'VIKN53907', 'MNSV53911', 'IXGP53982', 'XPLS45081', 'UWGO53377',
       'KVWJ40778', 'KBLG86592', 'APFU85474', 'CCQW73335', 'DXJZ75064',
       'JDMV76187', 'RJUL81287', 'TNXB83665', 'CIKE94068', 'HGUJ21007',
       'ZXFT21237', 'KBOK20943', 'MENA20785', 'CQAQ20361', 'HXJD21466',
       'GSNC20904', 'FRDZ21133', 'CNHG19808', 'DJDL21105', 'ABCB20467',
       'GLMK21133', 'VSPW20405', 'HTEZ21817', 'FVTQ20181', 'DCPP20168',
       'TXWS20511', 'XNOG20357', 'VSPW20414', 'PNST20989', 'EFTF21179',
       'ZZGN21450', 'KBOK20958', 'NQUG25995', 'WIZE26428', 'EDGX26018',
       'MHXR26588', 'YULB32421', 'XYHC31118', 'SJIT31691', 'ZZRV30730',
       'ITMG31457', 'NWXH31984', 'KRJP31426', 'BOWK32167', 'SDNY53354',
       'UQFO31564', 'SCKP53673', 'SCVF31712', 'MLVG32506', 'XDTG30724',
       'WGMG31517', 'PIJL35032', 'TSVR34841', 'HXJD35755', 'MOPJ

In [13]:
counter = 0
json_list = []

for item in codes:
    url = f"https://api.mercadolibre.com/inventories/{item}/stock/fulfillment"

    payload = {}
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    print(f'Buscando dados {counter}/{len(codes)}: {item}')

    try:
        response = requests.get(url, headers=headers, data=payload)
        response.raise_for_status()  # Lança uma exceção HTTPError para códigos de status de erro
        response_data = response.json()
        json_list.append(response_data)
        counter += 1

        if counter % 50 == 0:
            print(f"Fazendo uma pausa de 1 minuto...")
            time.sleep(60)

    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a requisição para {url}: {e}")

    except Exception as e:
        print(f"Erro não esperado: {e}")

# Continuar com o restante do código, se necessário


Buscando dados 0/596: JFGN34621
Buscando dados 1/596: ERIM51807
Buscando dados 2/596: HVCS52763
Buscando dados 3/596: ZQIV51979
Buscando dados 4/596: RBUB53192
Buscando dados 5/596: FXMN50419
Buscando dados 6/596: TDFV51283
Buscando dados 7/596: PKNN52962
Buscando dados 8/596: MAEG54107
Buscando dados 9/596: RYWV54124
Buscando dados 10/596: VIKN53907
Buscando dados 11/596: MNSV53911
Buscando dados 12/596: IXGP53982
Buscando dados 13/596: XPLS45081
Buscando dados 14/596: UWGO53377
Buscando dados 15/596: KVWJ40778
Buscando dados 16/596: KBLG86592
Buscando dados 17/596: APFU85474
Buscando dados 18/596: CCQW73335
Buscando dados 19/596: DXJZ75064
Buscando dados 20/596: JDMV76187
Buscando dados 21/596: RJUL81287
Buscando dados 22/596: TNXB83665
Buscando dados 23/596: CIKE94068
Buscando dados 24/596: HGUJ21007
Buscando dados 25/596: ZXFT21237
Buscando dados 26/596: KBOK20943
Buscando dados 27/596: MENA20785
Buscando dados 28/596: CQAQ20361
Buscando dados 29/596: HXJD21466
Buscando dados 30/59

In [14]:
len(json_list)

596

In [15]:
print(json_list)

[{'inventory_id': 'JFGN34621', 'total': 67, 'available_quantity': 67, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item', 'id': 'MLB924922735'}]}, {'inventory_id': 'ERIM51807', 'total': 32, 'available_quantity': 32, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item', 'id': 'MLB949788598'}]}, {'inventory_id': 'HVCS52763', 'total': 2, 'available_quantity': 0, 'not_available_quantity': 2, 'not_available_detail': [{'status': 'transfer', 'quantity': 2}], 'external_references': [{'type': 'item', 'id': 'MLB950214971'}]}, {'inventory_id': 'ZQIV51979', 'total': 0, 'available_quantity': 0, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item', 'id': 'MLB950295475'}]}, {'inventory_id': 'RBUB53192', 'total': 17, 'available_quantity': 17, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item', 'id': 'MLB950295955'}]}, {'inve

In [16]:
df = pd.DataFrame(json_list)
df

,inventory_id,total,available_quantity,not_available_quantity,not_available_detail,external_references
0,JFGN34621,67,67,0,[],"[{'type': 'item', 'id': 'MLB924922735'}]"
1,ERIM51807,32,32,0,[],"[{'type': 'item', 'id': 'MLB949788598'}]"
2,HVCS52763,2,0,2,"[{'status': 'transfer', 'quantity': 2}]","[{'type': 'item', 'id': 'MLB950214971'}]"
3,ZQIV51979,0,0,0,[],"[{'type': 'item', 'id': 'MLB950295475'}]"
4,RBUB53192,17,17,0,[],"[{'type': 'item', 'id': 'MLB950295955'}]"
...,...,...,...,...,...,...
591,RKDM61361,13,13,0,[],"[{'type': 'item', 'id': 'MLB2986538160', 'vari..."
592,PPWL59058,9,9,0,[],"[{'type': 'item', 'id': 'MLB2986538160', 'vari..."
593,RXXA58942,132,132,0,[],"[{'type': 'item', 'id': 'MLB2986538160', 'vari..."
594,POTW78210,2,2,0,[],"[{'type': 'item', 'id': 'MLB3106901850'}]"


In [17]:
df_full = df[['inventory_id', 'available_quantity']]
df_full

,inventory_id,available_quantity
0,JFGN34621,67
1,ERIM51807,32
2,HVCS52763,0
3,ZQIV51979,0
4,RBUB53192,17
...,...,...
591,RKDM61361,13
592,PPWL59058,9
593,RXXA58942,132
594,POTW78210,2


In [18]:
### AMBIENTE DE PRODUÇÃO ###
### Adicionando dados de estoque
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

for index, row in df_full.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO fulfillment_stock (ml_inventory_id, available_quantity)
        VALUES (%s, %s)
    """)
    cursor.execute(insert_query, (row['inventory_id'], row['available_quantity']))

conn.commit()

cursor.close()
conn.close()

print('Dados inseridos com sucesso!')

Dados inseridos com sucesso!


In [19]:
df_full

,inventory_id,available_quantity
0,JFGN34621,67
1,ERIM51807,32
2,HVCS52763,0
3,ZQIV51979,0
4,RBUB53192,17
...,...,...
591,RKDM61361,13
592,PPWL59058,9
593,RXXA58942,132
594,POTW78210,2


In [23]:
### AMBIENTE DE TESTES ###
### Repetindo os dados com dias diferentes
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
import datetime  

custom_created_at = datetime.datetime(2023, 11, 8, 1, 0, 0)  

for _ in range(30): # Repetir 30 vezes para adicionar dados com datas diferentes
    for index, row in df_full.iterrows(): # preenchendo db com dados de estoque
            insert_query = sql.SQL("""
                INSERT INTO fulfillment_stock (ml_inventory_id, available_quantity, created_at)
                VALUES (%s, %s, %s)
            """)
            cursor.execute(insert_query, (row['inventory_id'], row['available_quantity'], custom_created_at))
    custom_created_at += datetime.timedelta(days=-1)


conn.commit()

cursor.close()
conn.close()

print('Dados inseridos com sucesso!')

Dados inseridos com sucesso!


## É preciso o restante?

In [ ]:
df_er = json_normalize(json_list, record_path='external_references', meta=['inventory_id', 'total', 'available_quantity', 'not_available_quantity', 'not_available_detail'])
df_nad = json_normalize(json_list, record_path='not_available_detail', meta=['inventory_id', 'total', 'available_quantity', 'not_available_quantity', 'external_references'])


print('df',df.shape)
print('df_er',df_er.shape)
print('df_nad',df_nad.shape)

df_er.head(3)

In [ ]:
df_nad.head(3)

In [ ]:
print(df_nad.columns)
print(df_er.columns)

In [ ]:
df_nad_ = df_nad.drop(columns='external_references')
df_er_ = df_er.drop(columns='not_available_detail')

In [ ]:
common_cols = ['inventory_id', 'total', 'available_quantity', 'not_available_quantity']
# df_fulfillment = df_er_.merge(df_nad_, on=common_cols, how='left', suffixes=('_er','_nad'))
df_fulfillment = df_er_.merge(df_nad_, on=common_cols, how='left')

df_fulfillment.shape

In [ ]:
df_fulfillment.head(1)

In [ ]:
map_cols = {'inventory_id': 'ml_inventory_id', 'id': 'ml_item_id', 'status': 'nad_status','quantity':'nad_quantity'}
df_fulfillment = df_fulfillment.rename(columns=map_cols)
order_col = ['ml_inventory_id','ml_item_id','variation_id','nad_status','nad_quantity','total','available_quantity','not_available_quantity','type']
df_fulfillment = df_fulfillment[order_col]

df_fulfillment.head(1)

In [ ]:
df_fulfillment.shape

In [ ]:
df_fulfillment.info()

In [ ]:
df_fulfillment['variation_id'] = df_fulfillment['variation_id'].astype(str)
df_fulfillment['nad_quantity'] = df_fulfillment['nad_quantity'].fillna(0).astype(int)

In [ ]:
df_fulfillment.info()

In [ ]:
import psycopg2
from psycopg2 import sql

# Suponha que você já tenha seu DataFrame df_fulfillment

# Conecte-se ao banco de dados
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Itere pelas linhas do DataFrame e insira os dados na tabela
for index, row in df_fulfillment.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO ml_fulfillment3 (ml_inventory_id, ml_item_id, variation_id, nad_status, nad_quantity, total, available_quantity, not_available_quantity, type)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """)
    cursor.execute(insert_query, (row['ml_inventory_id'], row['ml_item_id'], row['variation_id'], row['nad_status'], row['nad_quantity'], row['total'], row['available_quantity'], row['not_available_quantity'], row['type']))

# Confirme as alterações
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()

print('Dados inseridos com sucesso!')


## Formatando

In [ ]:

# pegando codigos
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)

codes = df_codes['ml_inventory_id'].unique()

counter = 0
json_list = []

# diminuir codes

for item in codes:
    url = f"https://api.mercadolibre.com/inventories/{item}/stock/fulfillment"

    payload = {}
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    print(f'Buscando dados de: {item}')

    response = requests.get(url, headers=headers, data=payload)
    response_data = response.json()

    json_list.append(response_data)
    
    counter += 1
    
    if counter % 50 == 0:
        print(f"Fazendo uma pausa de 1 minuto...")
        time.sleep(60)
